In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [107]:
df = pd.read_csv('data/single_step_df_ints_2022-06-03_encoded.csv', header=0)

In [108]:
df

input target  start_coords.x  start_coords.y  start_coords.z  \
0     <start>      p       -0.451354       -0.413918        0.156247   
1           p      o        0.513000       -0.531000        0.740000   
2           o      c        0.513000       -0.531000        0.740000   
3           c      g        0.513000       -0.531000        0.740000   
4           g      k        0.513000       -0.531000        0.740000   
...       ...    ...             ...             ...             ...   
2069  <start>      d        1.000000        0.000000        2.000000   
2070        d      i        0.000000        1.500000        2.000000   
2071        i      k        0.000000        1.500000        2.000000   
2072        k      x        0.000000        1.500000        2.000000   
2073        x      q        0.000000        1.500000        2.000000   

      p.already_seen  n.already_seen  w.already_seen  t.already_seen  \
0                0.0             0.0             0.0             0.0   
1                1.0             0.0             0.0             0.0   
2                1.0             0.0             0.0             0.0   
3                1.0             0.0             0.0             0.0   
4                1.0             0.0             0.0             0.0   
...              ...             ...             ...             ...   
2069             0.0             0.0             0.0             0.0   
2070             0.0             0.0             0.0             0.0   
2071             0.0             0.0             0.0             0.0   
2072             0.0             0.0             0.0             0.0   
2073             0.0             0.0             0.0             0.0   

      g.already_seen  ...  v.food_k  v.strong_k  v.mid_k  coordinates_y.x  \
0                0.0  ...       NaN         NaN      NaN              NaN   
1                0.0  ...       NaN         NaN      NaN              NaN   
2                0.0  ...       NaN         NaN      NaN              NaN   
3                0.0  ...       NaN         NaN      NaN              NaN   
4                1.0  ...       NaN         NaN      NaN              NaN   
...              ...  ...       ...         ...      ...              ...   
2069             0.0  ...       NaN         NaN      NaN              NaN   
2070             0.0  ...       NaN         NaN      NaN              NaN   
2071             0.0  ...       NaN         NaN      NaN              NaN   
2072             0.0  ...       NaN         NaN      NaN              NaN   
2073             0.0  ...       NaN         NaN      NaN              NaN   

      coordinates_y.y  coordinates_y.z  y.containment  y.food_k  y.strong_k  \
0                 NaN              NaN            NaN       NaN         NaN   
1                 NaN              NaN            NaN       NaN         NaN   
2                 NaN              NaN            NaN       NaN         NaN   
3                 NaN              NaN            NaN       NaN         NaN   
4                 NaN              NaN            NaN       NaN         NaN   
...               ...              ...            ...       ...         ...   
2069              NaN              NaN            NaN       NaN         NaN   
2070              NaN              NaN            NaN       NaN         NaN   
2071              NaN              NaN            NaN       NaN         NaN   
2072              NaN              NaN            NaN       NaN         NaN   
2073              NaN              NaN            NaN       NaN         NaN   

      y.mid_k  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
...       ...  
2069      NaN  
2070      NaN  
2071      NaN  
2072      NaN  
2073      NaN  

[2074 rows x 197 columns]

In [109]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [110]:
df['target'].unique()
df['target'].value_counts()

k    278
p    249
f    172
g    168
d    164
c    152
s    142
b    110
i    106
w     74
a     74
o     73
r     60
u     56
q     43
v     30
h     28
e     22
t     22
z     21
m     13
n      9
x      7
y      1
Name: target, dtype: int64

In [111]:
index_to_drop = df['target'].loc[df['target'] == 'y'].index[0]
index_to_drop

1945

In [112]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [113]:
# remove row with target that only occurs once

df_new = df.drop(index=index_to_drop, axis=0)
df_new.reset_index(inplace=True, drop=True)

In [114]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new, df_new['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [115]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [116]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [42]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [43]:
batch_size = 128

In [117]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_3755/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}


In [45]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [46]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [47]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [118]:
all_inputs, encoded_features = create_input_data(df)

In [119]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [120]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(23)(x)

model = tf.keras.Model(all_inputs, output)

In [121]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [122]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
12/12 [==============================] - 7s 194ms/step - loss: 2.9149 - accuracy: 0.1365 - val_loss: 2.3313 - val_accuracy: 0.2026
Epoch 2/100
12/12 [==============================] - 0s 13ms/step - loss: 2.4088 - accuracy: 0.1916 - val_loss: 2.0909 - val_accuracy: 0.3151
Epoch 3/100
12/12 [==============================] - 0s 13ms/step - loss: 2.1927 - accuracy: 0.2309 - val_loss: 1.9830 - val_accuracy: 0.3698
Epoch 4/100
12/12 [==============================] - 0s 13ms/step - loss: 2.0820 - accuracy: 0.2584 - val_loss: 1.9010 - val_accuracy: 0.4084
Epoch 5/100
12/12 [==============================] - 0s 14ms/step - loss: 2.0109 - accuracy: 0.2867 - val_loss: 1.8475 - val_accuracy: 0.4277
Epoch 6/100
12/12 [==============================] - 0s 13ms/step - loss: 1.9345 - accuracy: 0.3150 - val_loss: 1.7912 - val_accuracy: 0.4277
Epoch 7/100
12/12 [==============================] - 0s 14ms/step - loss: 1.8483 - accuracy: 0.3535 - val_loss: 1.7405 - val_accuracy: 0.4341
Epoch

12/12 [==============================] - 0s 15ms/step - loss: 0.9927 - accuracy: 0.6382 - val_loss: 1.3358 - val_accuracy: 0.5627
Epoch 59/100
12/12 [==============================] - 0s 13ms/step - loss: 0.9677 - accuracy: 0.6437 - val_loss: 1.3521 - val_accuracy: 0.5595
Epoch 60/100
12/12 [==============================] - 0s 13ms/step - loss: 0.9813 - accuracy: 0.6513 - val_loss: 1.3536 - val_accuracy: 0.5595
Epoch 61/100
12/12 [==============================] - 0s 14ms/step - loss: 0.9808 - accuracy: 0.6409 - val_loss: 1.3576 - val_accuracy: 0.5691
Epoch 62/100
12/12 [==============================] - 0s 14ms/step - loss: 0.9612 - accuracy: 0.6589 - val_loss: 1.3571 - val_accuracy: 0.5756
Epoch 63/100
12/12 [==============================] - 0s 14ms/step - loss: 0.9799 - accuracy: 0.6340 - val_loss: 1.3811 - val_accuracy: 0.5531
Epoch 64/100
12/12 [==============================] - 0s 13ms/step - loss: 0.9923 - accuracy: 0.6340 - val_loss: 1.3569 - val_accuracy: 0.5820
Epoch 65/100

In [123]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

3/3 [==============================] - 0s 7ms/step - loss: 1.4880 - accuracy: 0.5370
Accuracy 0.5369774699211121


In [28]:
model.save('models/next_obj_classifier_2022-03-23')

2022-03-23 12:41:32.131340: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


In [31]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_2022-03-23')

In [ ]:
# test prediction for one sample

In [25]:
sample = df.loc[0].drop('target').to_dict()

In [26]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [27]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

NameError: name 'reloaded_model' is not defined

In [35]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [28]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(0, len(dataframe)): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [31]:
errors = get_prequential_error(df, model, labels, nr_of_sequences)

In [32]:
summed_error = [sum(error) for error in errors[:-1]]

In [33]:
np.median(summed_error)

2.0

In [34]:
#with open('results/nn_spatialinfo_prequential_summed_2022-04-14.txt', 'w') as file:
#    file.write(str(summed_error))